In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = None #2048 # Choose any! We auto support RoPE Scaling internally!
dtype = getattr(torch, "float16") # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


In [ ]:
model_id="mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    # max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


# Add LoRA adapters so we only need to update 1 to 10% of all parameters!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","lm_head"],
    lora_alpha = 16,
    lora_dropout = 0.1, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


In [ ]:
model

In [ ]:
tokenizer

In [ ]:
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer

# Data Preparation

In [ ]:
from datasets import load_dataset


In [ ]:
dataset = load_dataset('csv', data_files='/content/train.csv', split='train')
dataset

In [ ]:
# Split the dataset into train and valid sets
train_valid_split = dataset.train_test_split(test_size=0.1)

In [ ]:
# Get the train, validation, and test sets
train_dataset = train_valid_split['train']
valid_dataset = train_valid_split['test']

In [ ]:
train_dataset

In [ ]:
valid_dataset

In [ ]:
EOS_TOKEN = tokenizer.eos_token

train_prompt = """Below is a description for a time series data. Write a response that gives the name of the best fitting machine learning algorithm in one word without explanation.
The best algorithm name should be one of this search space algorithms: AdaboostRegressor, ElasticNetRegressor,  ExtraTreesRegressor,  LassoRegressor,  LightgbmRegressor, SVR, GaussianProcessRegressor, RandomForestRegressor or  XGBoostRegressor.

### DESCRIPTION:
{}

### RESPONSE:
{}"""

def formatting_prompts_func(examples):
    inputs       = examples["series_description"]
    outputs      = examples["algorithm"]
    texts = []
    for input, output in zip( inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = train_prompt.format( input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

In [ ]:
train_dataset

In [ ]:
train_dataset['text'][0]

In [ ]:
valid_dataset = valid_dataset.map(formatting_prompts_func, batched = True)
valid_dataset

In [ ]:
valid_dataset['text'][0]

# Model Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [ ]:
training_arguments= TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 32//4,
        gradient_checkpointing=True,
        warmup_steps = 5,
        max_steps = -1, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "./mistral_outputs",
        evaluation_strategy="steps", #epoch
        save_strategy="epoch",
    )

In [ ]:
from trl import  DataCollatorForCompletionOnlyLM

In [ ]:
instruction_template="DESCRIPTION:"
response_template = "RESPONSE:"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_arguments,
    data_collator =  DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,
                                                     response_template=response_template,
                                                     tokenizer=tokenizer,mlm=False),

)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
import time

In [ ]:
start= time.time()
trainer_stats = trainer.train()
print((time.time()-start)/60)

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# inference

In [ ]:
test_prompt = """Below is a description for a time series data. Write a response that gives the name of the best fitting machine learning algorithm in one word without explanation.
The best algorithm name should be one of this search space algorithms: AdaboostRegressor, ElasticNetRegressor,  ExtraTreesRegressor,  LassoRegressor,  LightgbmRegressor, SVR, GaussianProcessRegressor, RandomForestRegressor or  XGBoostRegressor.

### DESCRIPTION:
{}

### RESPONSE:
"""

def formatting_test_prompts_func(examples):
    inputs       = examples["series_description"]
    # outputs      = examples["algorithm"]
    texts = []
    for input, output in zip( inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = train_prompt.format( input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
test_dataset= load_dataset('csv', data_files='/content/test.csv')
test_dataset=test_dataset ['train']

In [ ]:
test_dataset = test_dataset.map(formatting_test_prompts_func, batched = True)
test_dataset

In [ ]:
test_dataset['text'][0]

In [ ]:
test_dataset['algorithm'][0]

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[test_dataset['text'][0]], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
test_dataset['algorithm'][1]

In [ ]:
inputs = tokenizer(
[test_dataset['text'][1]], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
test_responses=[]
# get all test data inference result
for test_prompt in test_dataset['text']:
  inputs= tokenizer(
  [test_prompt], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 10, use_cache = True)
  test_responses.append(tokenizer.batch_decode(outputs))


In [ ]:
test_responses

In [ ]:
# Convert the dataset to a Pandas DataFrame
df = test_dataset.to_pandas()

In [ ]:
df['model_responses']= test_responses

In [ ]:
# Save the DataFrame as a CSV file
df.to_csv('test_model_result_unsloth.csv', index=False)

In [ ]:
df

In [ ]:
predictions=[]
for response in test_responses:
  predictions.append( response[0].split('\n\n### RESPONSE:\n')[1].split('</s>')[0].strip())

predictions

In [ ]:
len(predictions)

In [ ]:
actual_data= df['algorithm']
len(actual_data)

In [ ]:
actual_data

In [ ]:
accuracy = sum(1 for true, pred in zip(actual_data, predictions) if true == pred) / len(actual_data)
accuracy

# save tuned model
To save the final model as LoRA adapters

In [ ]:
# Local saving
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")


In [ ]:
secret_hf = ''
!huggingface-cli login --token $secret_hf

In [ ]:
# Online saving on HF
new_model_adabtor= "unsloth-mistral-tuned"
model.push_to_hub(new_model_adabtor)
tokenizer.push_to_hub(new_model_adabtor)

In [ ]:
# Save and Merge to 4bit
model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

In [ ]:
# Save just LoRA adapters
model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

#GGUF / llama.cpp Conversion

Use **save_pretrained_ggu**f for local saving and **push_to_hub_ggu**f for uploading to HF.

Some supported quant methods:

- q8_0 - Fast conversion. High resource use, but generally acceptable.
- q4_k_m - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
- q5_k_m - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
model.push_to_hub_gguf("unsloth-mistral-ft-gguf", tokenizer, quantization_method = "q4_k_m", token="")

In [ ]:
model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")